# Cuaderno 1: Interfaz Python de Gurobi - Introducción

Describiremos en este cuaderno el uso básico de la interfaz Python de Gurobi para crear y resolver un modelo de de programación lineal entera.

Implementaremos el siguiente modelo, tomado del ejemplo [mip1.py](https://www.gurobi.com/documentation/9.5/examples/mip1_py.html#subsubsection:mip1.py) incluido en la documentación oficial de Gurobi.

\begin{align*}
&\max x + y + 2z\\
&\mbox{s.r.}\\
& x + 2y + 3z \leq 4\\
& x + y \geq 1\\
& x,y,z \in \{0, 1\}
\end{align*}

Lo primero que debemos hacer es importar el módulo `gurobipy` con las clases y funciones de la interfaz de Gurobi. Usaremos el acrónimo `gp` para referirnos a este módulo a lo largo del curso. Importaremos además el nombre `GRB` que contiene algunas constantes y tipos de uso frecuente:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

Luego llamamos al constructor `Model(<nombre_del_modelo>)` para crear un nuevo objeto que representará al modelo de programación lineal entera.

In [ ]:
m = gp.Model("mip1")
print(type(m))

La construcción, modificación y solución de este modelo se realiza a través de métodos de la clase Model.

Lo primero que debemos hacer es agregar variables al modelo. Para ello se puede emplear el método `addVar()`. Dos argumentos importantes que se pueden pasar a este método son `vtype`, que indica el tipo de la variable; y `name` que almacena su nombre como una cadena de caracteres.

Los valores posibles para `vtype` son `GRB.BINARY` (variables binarias), `GRB.INTEGER` (variables enteras no negativas) y `GRB.CONTINUOUS` (variables reales no negativas). 

In [ ]:
# Definir a x,y,z, como variables binarias
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

In [ ]:
print(type(x))

El siguiente paso es construir la función objetivo del modelo llamando al método `setObjective`. Este método recibe dos parámetros: una *expresión lineal* con la función objetivo en sí, y una constante que indica el sentido de la optimización. Para el segundo parámetro debe pasarse uno de los dos valores: `GRB.MAXIMIZE` o `GRB.MINIMIZE`.

In [ ]:
# Definir la función objetivo e indicar que se trata de maximizacion
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

Observar más de cerca el primer parámetro: la interfaz Python de Gurobi define un nuevo tipo de datos específico para construir **expresiones lineales**. Una expresión lineal se compone de una suma o resta de *términos lineales*. Cada término lineal puede ser una variable del modelo (ej: `x`), una expresión númerica, o el producto de una expresión númerica con una variable del modelo (ej: `2 * z`).

Una vez definida la función objetivo del modelo, el paso siguiente es agregar las restricciones llamando al método `addConstr()`:

In [ ]:
# Agregar dos restricciones al modelo
c0 = m.addConstr(x + 2 * y + 3 * z <= 4, "c0")
c1 = m.addConstr(x + y >= 1, "c1")
print(type(c0))

El primer parámetro de la función `addConstr()` es una restricción lineal, el segundo parámetro es una cadena de caracteres que almacena el nombre de la restricción.

La interfaz Python de Gurobi define un nuevo tipo de datos específico para representar **restricciones lineales**. Una expresión de este tipo se construye a partir de una expresión lineal; seguida por uno de los tres operadores de comparación `<=`, `==`, o `>=`; seguida por un número o por una segunda expresión lineal.

Para resolver el modelo, se llama al método `optimize()`:

In [ ]:
# Optimizar el modelo
m.optimize()

Al llamar a `optimize()`, se invocan automáticamente las funciones de la biblioteca de Gurobi necesarias para resolver el modelo. Se muestra la salida de las mismas.

Una vez terminada la solución del modelo, puede consultarse el valor óptimo de la función objetivo (en caso de que el modelo haya sido resuelto hasta la optimalidad) consultando el valor de `objVal`. Por otra parte, el método `m.getVars()` retorna un iterable con las variables del modelo; es posible emplear un lazo para consultar información como el valor de cada variable:

In [ ]:
# Valor de la funcion objetivo
print('Valor óptimo: {}'.format(m.objVal))
# Datos de las variables
print('Datos de las variables')
for v in m.getVars():
    print('Nombre: {}\t\tValor: {}'.format(v.varName, v.x))
    
print('Valor de x: {}'.format(x.x))
print('Valor de y: {}'.format(y.x))
print('Valor de z: {}'.format(z.x))

## Más información

El solver Gurobi viene acompañado de amplia documentación. Pueden consultarse, por ejemplo:

* Las [guías de acceso rápido (Quick Start Guides)](https://www.gurobi.com/documentation/quickstart.html), para instrucciones acerca de la instalación, configuración y primeros pasos en el uso de Gurobi.
* Los [manuales de referencia (Reference Manuals)](https://www.gurobi.com/documentation/10.0/refman/index.html), para una descripción detallada de las diferentes funciones del solver y de sus APIs a varios lenguajes de programación.
* [Códigos de ejemplo](https://www.gurobi.com/documentation/10.0/examples/python_examples.html), en varios lenguajes de programación, que ilustran las diferentes características del solver, a través de la implementación de algunos modelos de programación lineal entera. 
* [Cuadernos de Jupyter](https://www.gurobi.com/resource/modeling-examples-using-the-gurobi-python-api-in-jupyter-notebook/) con ejemplos de la implementación de modelos de optimización basados en problemas reales, que ilustran la aplicabilidad de la optimización matemática en una gran variedad industrias. Los ejemplos están adptados del libro "*Model Building in Mathematical Programming*" (H. Paul Williams).



## Código completo

Se reproduce a continuación el código completo del modelo anterior, tal como se encuentra documentado en el manual de Gurobi.

Observar que las instrucciones anteriores están contenidas dentro de un bloque `try` ... `except`. Esta es una práctica recomendada para el correcto manejo de las excepciones que pudieran producirse durante la construcción y solución del modelo.

In [ ]:
#!/usr/bin/env python3.7

# Copyright 2022, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary

import gurobipy as gp
from gurobipy import GRB

try:

    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    # Valor de la función objetivo
    print('Valor óptimo: {}'.format(m.objVal))
    
    # Datos de las variables
    print('Datos de las variables')
    for v in m.getVars():
        print('Nombre: {}\t\tValor: {}'.format(v.varName, v.x))
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')